In [1]:
# Install dependencies
import kagglehub

from kagglehub import KaggleDatasetAdapter

from datasets import load_dataset

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import MultiOutputClassifier

import pandas as pd

import joblib

C:\Users\s050741159\Desktop\hackathon-sentimentapi-analytics\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = 'olist_order_reviews_dataset.csv'

# Load the latest version
df_olist = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "olistbr/brazilian-ecommerce", file_path, pandas_kwargs={"encoding":'utf-8', "quotechar":'"', "on_bad_lines":'skip'})

hf = load_dataset("ruanchaves/b2w-reviews01")

df_b2w = pd.concat([hf['train'].to_pandas()], ignore_index=True)

In [3]:
df_b2w.drop(axis="columns", columns=(['submission_date', 'reviewer_id', 'product_id', 'product_name',
       'product_brand', 'site_category_lv1', 'site_category_lv2',
       'review_title', 'reviewer_birth_year', 'reviewer_gender', 'reviewer_state', 'recommend_to_a_friend']), inplace=True)

In [4]:
df_olist.drop(axis="columns", columns=(['review_id', 'order_id','review_creation_date',
       'review_answer_timestamp', 'review_comment_title']), inplace=True)

In [5]:
df_b2w.dropna(subset=['review_text'], inplace=True)

df_olist.dropna(subset=['review_comment_message'], inplace=True)

In [6]:
mapa_sentimentos = {
    1: 'Negativo',
    2: 'Negativo',
    3: 'Neutro',
    4: 'Positivo',
    5: 'Positivo'
}

df_b2w['sentimento'] = df_b2w['overall_rating'].map(mapa_sentimentos)
df_olist['sentimento'] = df_olist['review_score'].map(mapa_sentimentos)

df_olist.drop(axis="columns", columns=(['review_score']), inplace=True)

df_b2w.drop(axis="columns", columns=(['overall_rating']), inplace=True)

In [7]:
df_olist.columns = ['texto', 'sentimento']
df_b2w.columns = ['texto', 'sentimento']

In [10]:
# Agora, concatenamos os dois datasets, ao mesmo tempo em que resetamos os index e os "embaralhamos"

# 1. Concatenar os dois dataframes (empilhar um sobre o outro)
# ignore_index=True ajuda a não duplicar índices antigos, mas o reset abaixo garante a limpeza final
df_pt = pd.concat([df_olist, df_b2w], ignore_index=True)

# 2. Embaralhar as linhas
# frac=1 significa "retorne 100% dos dados", mas em ordem aleatória
# reset_index(drop=True) recria o índice de 0 a N, para não ficar tudo bagunçado (ex: 5, 100, 2...)
df_pt = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

In [11]:
df_pt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170075 entries, 0 to 170074
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   texto       170075 non-null  object
 1   sentimento  170075 non-null  object
dtypes: object(2)
memory usage: 2.6+ MB


In [12]:
dataset = load_dataset("mteb/amazon_reviews_multi", "es")

df_es = pd.concat([dataset['train'].to_pandas(), dataset['test'].to_pandas(), dataset['validation'].to_pandas()], ignore_index=True)

In [13]:
# DF_ES:
# 0,1 => Negativo - 2 => Neutro - 3,4 => Positivo. Criamos uma nova coluna com esse "Mapeamento" usando a função .map()

mapa_sentimentos = {
    0: 'Negativo',
    1: 'Negativo',
    2: 'Neutro',
    3: 'Positivo',
    4: 'Positivo'
}

df_es['sentimento'] = df_es['label'].map(mapa_sentimentos)

In [14]:
df_es.drop(axis="columns", columns=['id', 'label', 'label_text'], inplace=True)
df_es.columns = ['texto', 'sentimento']

In [15]:
df_pt['idioma']="pt"
df_es['idioma']='es'

In [16]:
# 1. Concatena
df_multi = pd.concat([df_pt, df_es])

# 2. Embaralha e reseta o índice
# frac=1 -> Pega 100% das linhas aleatoriamente
# random_state=42 -> Garante que o embaralhamento seja sempre o mesmo (importante para reproduzir seus testes)
# reset_index(drop=True) -> Cria um índice novo (0, 1, 2...) e joga fora o antigo bagunçado

df_multi = df_multi.sample(frac=1, random_state=42).reset_index(drop=True)

In [17]:
df_multi.to_csv('../datasets/df_multi_concatenado.csv', encoding='utf-8',index=False, header=True, sep=';')

In [18]:
# X: textos; y: DataFrame com duas colunas
X = df_multi['texto']
y = df_multi[['sentimento', 'idioma']]

base_clf = LogisticRegression(solver='lbfgs', max_iter=1000)

modelo_multi = make_pipeline(
    TfidfVectorizer(max_features=10000,
                    strip_accents='unicode',
                    lowercase=True,
                    ngram_range=(1, 2)),
    MultiOutputClassifier(base_clf)
)
modelo_multi.fit(X, y)

print("⏳ Treinando modelo...")

# 3. Salvar
joblib.dump(modelo_multi, "../models/modelo_multi.joblib")
print("✅ Novo modelo MULTI salvo em: modelo_multi.joblib")

⏳ Treinando modelo...
✅ Novo modelo MULTI salvo em: modelo_multi.joblib
